<a href="https://colab.research.google.com/github/markiroberts/gogglecolab01/blob/main/03_pt02_Use_matches_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
import os
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import data_table
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

data_table.enable_dataframe_formatter()

directory_path = "/content/drive/MyDrive/ColabData/"
try:
  os.stat(directory_path)
except:
  print(f"Cant access {directory_path}")
  drive.mount('/content/drive')

match_df = pd.read_csv("/content/drive/MyDrive/ColabData/matches.csv", index_col=0)

match_df = match_df.sort_values(["season", "team", "date"])
match_df = match_d2.reset_index(drop=True)

print(match_df[["season", "team", "opponent","date"]].head())
print(match_df[["season", "team", "opponent","date"]].tail())

   season     team         opponent        date
0    2021  Arsenal        Brentford  2021-08-13
1    2021  Arsenal          Chelsea  2021-08-22
2    2021  Arsenal  Manchester City  2021-08-28
3    2021  Arsenal     Norwich City  2021-09-11
4    2021  Arsenal          Burnley  2021-09-18
      season                     team         opponent        date
1875    2023  Wolverhampton Wanderers          Arsenal  2023-12-02
1876    2023  Wolverhampton Wanderers          Burnley  2023-12-05
1877    2023  Wolverhampton Wanderers  Nott'ham Forest  2023-12-09
1878    2023  Wolverhampton Wanderers         West Ham  2023-12-17
1879    2023  Wolverhampton Wanderers          Chelsea  2023-12-24


In [288]:
def rolling_averages(group, cols):
  new_cols = []
  for x in cols:
    y = x + "_rolling"
    new_cols.append(y)
  group = group.sort_values("dateandtime")
  rolling_stats = group[cols].rolling(3, closed='left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group, new_cols


cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
match_rolling = rolling_averages(match_team, cols)
match_rolling

(      Unnamed: 0        date   time            comp         round  day   gf  \
 1427           3  2021-09-11  15:00  Premier League   Matchweek 4  Sat  2.0   
 1428           4  2021-09-18  12:30  Premier League   Matchweek 5  Sat  0.0   
 1429           5  2021-09-26  14:00  Premier League   Matchweek 6  Sun  1.0   
 1430           6  2021-10-02  15:00  Premier League   Matchweek 7  Sat  2.0   
 1431           7  2021-10-16  15:00  Premier League   Matchweek 8  Sat  3.0   
 ...          ...         ...    ...             ...           ...  ...  ...   
 357           13  2023-12-02  15:00  Premier League  Matchweek 14  Sat  1.0   
 358           14  2023-12-05  19:30  Premier League  Matchweek 15  Tue  1.0   
 359           15  2023-12-09  15:00  Premier League  Matchweek 16  Sat  1.0   
 360           16  2023-12-17  14:00  Premier League  Matchweek 17  Sun  0.0   
 361           17  2023-12-24  13:00  Premier League  Matchweek 18  Sun  2.0   
 
        ga         opponent   xg  ... 

In [278]:
#df = pd.DataFrame({'B': [0, 1, 2, 3, 4]})
#df['Bavg2'] = df.B.shift(1).rolling(window=2).mean()
#df = df.dropna(subset='Bavg2')
#https://fbref.com/en/comps/9/Premier-League-Stats

teamdict = {
"Wolverhampton Wanderers": "Wolves",
"Brighton and Hove Albion": "Brighton",
"West Ham United": "West Ham",
"Tottenham Hotspur": "Tottenham",
"Manchester United": "Manchester Utd",
"Newcastle United": "Newcastle Utd",
"Sheffield United": "Sheffield Utd",
"Nottingham Forest": "Nott'ham Forest"
}

def getTeam(team):
  opponent = teamdict.get(team,None)
  if opponent:
    return opponent
  else:
    return team


match_df = pd.read_csv("/content/drive/MyDrive/ColabData/matches.csv")
match_df.head()

# the team name an opponent list are initially not the same !?
# translate team to the equivalent from oponent list 'Sheffield United -> Sheffield Utd.' etc.
match_df['team'] = match_df.apply(lambda row: getTeam(row.team), axis=1)
teamset = sorted(set(match_df['team'].tolist()))
opponentset = sorted(set(match_df['opponent'].tolist()))
allmatch_team = pd.DataFrame()

for team in teamset:
  options = []
  options.append(team)

# selecting rows based on condition

  match_team = match_df[match_df['team'].isin(options)].copy()
#  match_team['result_text'] = match_team['result']
#  match_team['venue_text'] = match_team['venue']
  match_team["venue_code"] = match_team["venue"].astype("category").cat.codes
#  match_team['at_home'] = match_team.apply(lambda row: row.venue == 'Home', axis=1)
  match_team['result_text'] = match_team.apply(lambda row: row.result, axis=1)
#  match_team['home'] = match_team.apply(lambda row: row.venue, axis=1)
  selection = ['result','venue']
  match_team = pd.get_dummies(match_team, columns=selection)
#  match_team['venue_code'] = match_team["venue"].astype("category")
  match_team['dateandtime'] = pd.to_datetime( match_team['date'] + ' ' + match_team['time'])
  match_team['hour'] = match_team['dateandtime'].dt.hour
  match_team['result'] = 1.0 * match_team['result_W'] - 1.0 * match_team['result_L']
  match_team.sort_values("dateandtime")

  match_team['dayofweek'] = match_team.apply(lambda row: row.dateandtime.weekday(), axis=1)

#  match_team['result_1']    = match_team.result.shift(1).rolling(window=1).mean()
#  match_team['result_L_1']  = match_team.result_L.shift(1).rolling(window=1).mean()
#  match_team['result_D_1']  = match_team.result_D.shift(1).rolling(window=1).mean()
#  match_team['result_W_1']  = match_team.result_W.shift(1).rolling(window=1).mean()
#  match_team['gf_1']        = match_team.gf.shift(1).rolling(window=1).mean()
#  match_team['ga_1']        = match_team.ga.shift(1).rolling(window=1).mean()
#  match_team['sh_1']        = match_team.sh.shift(1).rolling(window=1).mean()
#  match_team['sot_1']       = match_team.sot.shift(1).rolling(window=1).mean()
#  match_team['fk_1']        = match_team.fk.shift(1).rolling(window=1).mean()
#  match_team['pk_1']        = match_team.pk.shift(1).rolling(window=1).mean()
#  match_team['pkatt_1']     = match_team.pkatt.shift(1).rolling(window=1).mean()
#  match_team['xg_1']        = match_team.xg.shift(1).rolling(window=1).mean()
#  match_team['xga_1']       = match_team.xga.shift(1).rolling(window=1).mean()
#  match_team['poss_1']      = match_team.poss.shift(1).rolling(window=1).mean()

#  match_team['result_L_3']  = match_team.result_L.shift(1).rolling(window=3).mean()
#  match_team['result_D_3']  = match_team.result_D.shift(1).rolling(window=3).mean()
#  match_team['result_W_3']  = match_team.result_W.shift(1).rolling(window=3).mean()
#  match_team['gf_3']        = match_team.gf.shift(1).rolling(window=3).mean()
#  match_team['ga_3']        = match_team.ga.shift(1).rolling(window=3).mean()
#  match_team['sh_3']        = match_team.ga.shift(1).rolling(window=1).mean()
#  match_team['sot_3']       = match_team.sot.shift(1).rolling(window=3).mean()
#  match_team['fk_3']        = match_team.fk.shift(1).rolling(window=3).mean()
#  match_team['pk_3']        = match_team.pk.shift(1).rolling(window=3).mean()
#  match_team['pkatt_3']     = match_team.pkatt.shift(1).rolling(window=3).mean()
#  match_team['xg_3']        = match_team.xg.shift(1).rolling(window=1).mean()
#  match_team['xga_3']       = match_team.xga.shift(1).rolling(window=1).mean()
#  match_team['poss_3']      = match_team.poss.shift(1).rolling(window=1).mean()

#  match_team['result_3']   = match_team.result.shift(1).rolling(window=3).mean()
#  match_team['result_L_5'] = match_team.result_L.shift(1).rolling(window=5).mean()
#  match_team['result_D_5'] = match_team.result_D.shift(1).rolling(window=5).mean()
#  match_team['result_W_5'] = match_team.result_W.shift(1).rolling(window=5).mean()
#  match_team['gf_5']        = match_team.gf.shift(1).rolling(window=5).mean()
#  match_team['ga_5']        = match_team.ga.shift(1).rolling(window=5).mean()
#  match_team['sh_5']        = match_team.sh.shift(1).rolling(window=5).mean()
#  match_team['sot_5']       = match_team.sot.shift(1).rolling(window=5).mean()
#  match_team['fk_5']        = match_team.fk.shift(1).rolling(window=5).mean()
#  match_team['pk_5']        = match_team.pk.shift(1).rolling(window=5).mean()
#  match_team['pkatt_5']     = match_team.pkatt.shift(1).rolling(window=5).mean()
#  match_team['xg_5']        = match_team.xg.shift(1).rolling(window=5).mean()
#  match_team['xga_5']       = match_team.xga.shift(1).rolling(window=5).mean()
#  match_team['poss_5']      = match_team.poss.shift(1).rolling(window=5).mean()
#  match_team['result_5']   = match_team.result.shift(1).rolling(window=5).mean()

  match_team = match_team.dropna()

  matchedcols = []
  columnlist = ['dateandtime','hour','season','result','result',  'sh', 'sot', 'gf', 'dist', 'ga', 'day', 'xg', 'hour', 'fk', 'pk', 'poss', 'team', 'opponent','season', 'venue_code', 'result_text']
  for findcol in match_team.columns:
    for matchcol in columnlist:
      if findcol.startswith(matchcol):
        if findcol not in matchedcols:
          matchedcols.append(findcol)
  newmatch_team = match_team[matchedcols]
  allmatch_team = pd.concat([allmatch_team,newmatch_team], ignore_index=True)


allmatch_team = allmatch_team.sort_values(by=['team','dateandtime'], ignore_index=True)
allmatch_team = allmatch_team.reset_index(drop=True)
allmatch_team["team_code"] = allmatch_team["team"].astype("category").cat.codes
allmatch_team["opp_code"] = allmatch_team["opponent"].astype("category").cat.codes
allmatch_team["target"] = (allmatch_team["result_text"] == "W").astype("int")

cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
allmatch_rolling = allmatch_team.groupby("team").apply(lambda x: rolling_averages(x, cols))
allmatch_rolling = allmatch_rolling.droplevel('team')
allmatch_rolling = allmatch_rolling.dropna()
allmatch_rolling.index = range(allmatch_rolling.shape[0])
allmatch_rolling["team"]

0       Arsenal
1       Arsenal
2       Arsenal
3       Arsenal
4       Arsenal
         ...   
1800     Wolves
1801     Wolves
1802     Wolves
1803     Wolves
1804     Wolves
Name: team, Length: 1805, dtype: object

In [286]:
def make_predictions(data, predictors):
  rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=42)
  train = data[data["dateandtime"] < '2023-07-01']
  test = data[data["dateandtime"] >= '2023-07-01']
  rf.fit( train[predictors], train["target"])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
  precision = precision_score(test["target"], preds)
  print( f"predictors: {predictors} | accuracy: {accuracy} | precision: {precision}" )
  return combined, precision

In [287]:
predictors = ["venue_code", "opp_code", "hour", "dayofweek"]
combined, precision = make_predictions(allmatch_rolling, predictors)

predictors: ['venue_code', 'opp_code', 'hour', 'dayofweek'] | accuracy: 0.5725190839694656 | precision: 0.4878048780487805


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=42)

In [230]:
preds = rf.predict(test[predictors])
accuracy = accuracy_score(test["target"], preds)
precision = precision_score(test["target"], preds)
print( f"predictors: {predictors} | accuracy: {accuracy} | precision: {precision}" )

predictors: ['venue_code', 'opp_code', 'hour', 'dayofweek'] | accuracy: 0.5725190839694656 | precision: 0.45588235294117646


In [221]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [227]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])
+31/68

0.45588235294117646